In [19]:
import random
import tensorflow as tf
import os
import numpy as np
def seed_tensorflow(seed=415):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_tensorflow(415)
from sklearn import metrics
import json
import datetime as dt
from core.data_processor import DataLoader
from core.utils import Timer
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch

In [20]:
class MyHyperModel(HyperModel):

    # 这里是个摆设
    def __init__(self):
        self.name = "HyperModel"

    def build(self, hp):
        configs = json.load(open('config_2.json', 'r'))
        model = tf.keras.models.Sequential()
        timer = Timer()
        timer.start()

        for layer in configs['model']['layers']:
            neurons = layer['neurons'] if 'neurons' in layer else None
            dropout_rate = layer['rate'] if 'rate' in layer else None
            activation = layer['activation'] if 'activation' in layer else None
            return_seq = layer['return_seq'] if 'return_seq' in layer else None
            input_timesteps = layer['input_timesteps'] if 'input_timesteps' in layer else None
            input_dim = layer['input_dim'] if 'input_dim' in layer else None

            # lstm参数范围
            if layer['type'] == 'lstm':
                neurons = hp.Int('units',min_value=64,max_value=256,step=2)

            if layer['type'] == 'dense':
                model.add(tf.keras.layers.Dense(neurons, activation=activation))
            if layer['type'] == 'lstm':
                model.add(tf.keras.layers.LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=return_seq))
            if layer['type'] == 'dropout':
                model.add(tf.keras.layers.Dropout(dropout_rate))

        model.compile(keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),loss=configs['model']['loss']) # 学习率选择

        print('[Model] Model Compiled')
        timer.stop()
        return model

In [21]:
#读取所需参数
configs = json.load(open('config_2.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])
    #读取数据
data = DataLoader(
    os.path.join('data', configs['data']['filename']),
    configs['data']['train_test_split'],
    configs['data']['columns']
)

x, y = data.get_train_data(
            seq_len=configs['data']['sequence_length'],
            normalise=configs['data']['normalise']
        )

length = x.shape[0]
val_split = int(-0.1765 * length) # 看的你的config里面的划分
val_x = x[val_split:]
val_y = y[val_split:]

In [23]:
hyperModel = MyHyperModel()

tuner = RandomSearch(
    hyperModel,
    objective='val_loss', # 最小化指标，可以多个
    max_trials=288,
    seed=415,
    project_name='myLSTM_try') # 保存在该文件夹下

[Model] Model Compiled
Time taken: 0:00:00.356481


In [26]:
tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))

Trial 277 Complete [00h 00m 50s]
val_loss: 0.007142418064177036

Best val_loss So Far: 0.004398881457746029
Total elapsed time: 02h 16m 11s
INFO:tensorflow:Oracle triggered exit
